In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [16]:
# Load the dataset
df = pd.read_csv("../data/data_properties.csv")

# Filter houses and apartments
df_house = df[df['type'] == 'HOUSE'].copy()
df_apartment = df[df['type'] == 'APARTMENT'].copy()


In [17]:

def create_features(df):

    df_house['price_per_sqm'] = df_house['price_main'] / df_house['surface']
    df_apartment['price_per_sqm'] = df_apartment['price_main'] / df_apartment['surface']
    
    region_avg_price = df.groupby('region')['price_per_sqm'].transform('mean')
    province_avg_price = df.groupby('province')['price_per_sqm'].transform('mean')
    df['region_avg_price'] = region_avg_price
    df['province_avg_price'] = province_avg_price
    
    # Size-related Features
    df['surface_to_surfaceGood_ratio'] = df['surface'] / df['surfaceGood']
        
    # Property Type-specific Features
    df['terrace_per_sqm'] = df['terraceSurface'] / df['surface']
    
    # Income-related Features
    df['cadastral_income_per_sqm'] = df['cadastralIncome'] / df['surface']
    
    # Energy-related Features
    df['epc_score_impact'] = df['epcScores'].astype('category').cat.codes * df['bedrooms']
    
    return df

# Apply feature engineering to df_house and df_apartment
df_house = create_features(df_house)
df_apartment = create_features(df_apartment)

# Display the updated dataframes
print("House Data:")
print(df_house.head())
print("\nApartment Data:")
print(df_apartment.head())


House Data:
          id    region         province district  locality  postalcode  \
0   10858284  Wallonie       Luxembourg    Arlon   Halanzy        6792   
2   11137665  Flanders    East Flanders     Gent  Ertvelde        9940   
3   10966577  Flanders          Antwerp  Antwerp  Kapellen        2950   
6   11045528  Flanders  Flemish Brabant   Leuven   Lubbeek        3210   
10  11061091  Wallonie            Liège    Liège   Seraing        4100   

     latitude  longitude  price_main   type  ...       floodZone  \
0   49.582632   5.756293    220000.0  HOUSE  ...             NaN   
2   51.179353   3.740274    825000.0  HOUSE  ...  NON_FLOOD_ZONE   
3   51.345060   4.401032    575000.0  HOUSE  ...  NON_FLOOD_ZONE   
6   50.878226   4.879963    439000.0  HOUSE  ...  NON_FLOOD_ZONE   
10  50.591924   5.508498    169000.0  HOUSE  ...             NaN   

   gardenSurface  isNewRealEstateProject price_per_sqm  region_avg_price  \
0           80.0                       0   1383.647799    

In [18]:
def check_missing_data(df, property_type):
    print(f"Missing data for {property_type}:")
    missing_data = df.isna().sum()
    percentage_missing = (missing_data / len(df)) * 100
    print(percentage_missing)
    return missing_data

# Check missing data for df_house
check_missing_data(df_house, "House")

# Check missing data for df_apartment
check_missing_data(df_apartment, "Apartment")

Missing data for House:
id                                   0.000000
region                               0.000000
province                             0.000000
district                             0.000000
locality                             0.000000
postalcode                           0.000000
latitude                             0.000000
longitude                            0.000000
price_main                           0.008501
type                                 0.000000
subtype                              0.000000
heatingType                         33.088694
cadastralIncome                      0.000000
epcScores                           28.024936
primaryEnergyConsumption            19.991499
bedrooms                             0.000000
surface                             15.715500
surfaceGood                          0.000000
hasGasWaterElectricityConnection    41.130632
condition                           29.113063
facadeCount                         20.017002
hasKitchen

id                                      0
region                                  0
province                                0
district                                0
locality                                0
postalcode                              0
latitude                                0
longitude                               0
price_main                              0
type                                    0
subtype                                 0
heatingType                          8771
cadastralIncome                         0
epcScores                            7099
primaryEnergyConsumption             6681
bedrooms                                0
surface                              2136
surfaceGood                         23803
hasGasWaterElectricityConnection    23803
condition                            7375
facadeCount                         10640
hasKitchenSetup                      9838
isFurnished                         15737
fireplaceExists                   

In [19]:
def drop_columns_with_missing_values(df, type):
    missing_data_all = check_missing_data(df,type)
    columns_to_drop = missing_data_all[missing_data_all > 50].index.tolist()
    df = df.drop(columns=columns_to_drop)
    return df

# Drop columns with more than 50 missing values from df_house
df_house = drop_columns_with_missing_values(df_house, 'HOUSE')

# Drop columns with more than 50 missing values from df_apartment
df_apartment = drop_columns_with_missing_values(df_apartment, 'APARTMENT')

print(df_apartment.columns)
print(df_house.columns)

Missing data for HOUSE:
id                                   0.000000
region                               0.000000
province                             0.000000
district                             0.000000
locality                             0.000000
postalcode                           0.000000
latitude                             0.000000
longitude                            0.000000
price_main                           0.008501
type                                 0.000000
subtype                              0.000000
heatingType                         33.088694
cadastralIncome                      0.000000
epcScores                           28.024936
primaryEnergyConsumption            19.991499
bedrooms                             0.000000
surface                             15.715500
surfaceGood                          0.000000
hasGasWaterElectricityConnection    41.130632
condition                           29.113063
facadeCount                         20.017002
hasKitchen

In [20]:
# Load the dataset
df = pd.read_csv("../data/data_properties.csv")

# Filter houses and apartments
df_house = df[df['type'] == 'HOUSE'].copy()
df_apartment = df[df['type'] == 'APARTMENT'].copy()

def cleaning_data(df, property_type):
    ## Handling Outliers
    Q1 = df['price_main'].quantile(0.25) # Calculate Q1
    Q3 = df['price_main'].quantile(0.75) # Calculate Q3

    IQR = Q3 - Q1 # Calculate IQ range

    lower_bound = Q1 - 1.5 * IQR # Calculate the lower bound
    upper_bound = Q3 + 1.5 * IQR # Calculate the upper bound

    df = df[(df['price_main'] >= lower_bound) & (df['price_main'] <= upper_bound)] # Filter rows where 'price_main' < lb or > ub

    # Drop unnecessary columns
    # columns_to_drop = ['cadastralIncome', 'id', 'primaryEnergyConsumption', 'isNewRealEstateProject', 'postalcode', 
    #                    'hasKitchenSetup', 'isFurnished', 'heatingType', 'floodZone', 'locality', 'latitude', 'longitude']
    
    # Drop columns only if they exist in the DataFrame
    # existing_columns = [col for col in columns_to_drop if col in df.columns]
    # df = df.drop(columns=existing_columns)

    missing_data_all = df.isna().sum()
    percentage_missing_all = round(missing_data_all * 100 / len(df), 0)

    # Drop columns with more than 50% missing values
    df.drop(columns=get_column_missing_values(percentage_missing_all), inplace=True)  # Modify DataFrame inplace
    
    print(f"{property_type} DataFrame shape after cleaning: {df.shape}")

    return df

def get_column_missing_values(percentage_missing_all):
    columns_to_drop = []
    for column, missing_percentage in percentage_missing_all.items():
        if missing_percentage >= 50:
            columns_to_drop.append(column)
    return columns_to_drop

# Cleaning data for houses
df_house= cleaning_data(df_house, "House")

# Cleaning data for apartments
df_apartment= cleaning_data(df_apartment, "Apartment")

print(df_apartment.columns)
print(df_house.columns)

House DataFrame shape after cleaning: (32944, 25)
Apartment DataFrame shape after cleaning: (21791, 23)
Index(['id', 'region', 'province', 'district', 'locality', 'postalcode',
       'latitude', 'longitude', 'price_main', 'type', 'subtype', 'heatingType',
       'cadastralIncome', 'epcScores', 'primaryEnergyConsumption', 'bedrooms',
       'surface', 'condition', 'facadeCount', 'hasKitchenSetup',
       'fireplaceExists', 'floodZone', 'isNewRealEstateProject'],
      dtype='object')
Index(['id', 'region', 'province', 'district', 'locality', 'postalcode',
       'latitude', 'longitude', 'price_main', 'type', 'subtype', 'heatingType',
       'cadastralIncome', 'epcScores', 'primaryEnergyConsumption', 'bedrooms',
       'surface', 'surfaceGood', 'hasGasWaterElectricityConnection',
       'condition', 'facadeCount', 'hasKitchenSetup', 'fireplaceExists',
       'floodZone', 'isNewRealEstateProject'],
      dtype='object')


C:\Users\arian\AppData\Local\Temp\ipykernel_16492\3802197773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=get_column_missing_values(percentage_missing_all), inplace=True)  # Modify DataFrame inplace
C:\Users\arian\AppData\Local\Temp\ipykernel_16492\3802197773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=get_column_missing_values(percentage_missing_all), inplace=True)  # Modify DataFrame inplace


In [21]:
def impute_missing_values(X_train, X_test):
    # Impute missing values for numerical features
    numerical_imputer = SimpleImputer(strategy='mean')
    numerical_features = X_train.select_dtypes(include=['int64', 'float64']).columns
    print(X_train[numerical_features].head(20))
    X_train[numerical_features] = numerical_imputer.fit_transform(X_train[numerical_features])
    X_test[numerical_features] = numerical_imputer.transform(X_test[numerical_features])

    # Impute missing values for categorical features
    categorical_imputer = SimpleImputer(strategy='most_frequent')
    categorical_features = X_train.select_dtypes(include=['object']).columns
    X_train[categorical_features] = categorical_imputer.fit_transform(X_train[categorical_features])
    X_test[categorical_features] = categorical_imputer.transform(X_test[categorical_features])

    return X_train, X_test

# Cleaning data for houses
df_house = cleaning_data(df_house, "House")

# Cleaning data for apartments
df_apartment = cleaning_data(df_apartment, "Apartment")

# Split houses data into training and testing sets
X_train_house, X_test_house = train_test_split(df_house.drop('price_main', axis=1), test_size=0.2, random_state=42)
y_train_house, y_test_house = train_test_split(df_house['price_main'], test_size=0.2, random_state=42)

# Impute missing values for houses
X_train_house, X_test_house = impute_missing_values(X_train_house, X_test_house)

# Split apartments data into training and testing sets
X_train_apartment, X_test_apartment = train_test_split(df_apartment.drop('price_main', axis=1), test_size=0.2, random_state=42)
y_train_apartment, y_test_apartment = train_test_split(df_apartment['price_main'], test_size=0.2, random_state=42)

# Impute missing values for apartments
X_train_apartment, X_test_apartment = impute_missing_values(X_train_apartment, X_test_apartment)

print(df_house.head())
print(df_apartment.head())

House DataFrame shape after cleaning: (32220, 25)
Apartment DataFrame shape after cleaning: (21232, 23)
             id  postalcode   latitude  longitude  cadastralIncome  \
9338   11079012        4032  50.614829   5.612938              664   
61289  11041339        4250  50.668047   5.187740              450   
14921  10805399        8450  51.252348   2.985051             1400   
13162  11136139        3500  50.967183   5.358411                0   
29224  11071911        2880  51.094303   4.239369                0   
39325  10973270        6280  50.338129   4.489946              620   
23297  11161124        9150  51.127826   4.292999                0   
55412  10547545        2000  51.219865   4.408535                0   
12344  11150412        3510  50.948274   5.266551                0   
36360  10939378        8790  50.851194   3.408053              302   
8698   11075539        3210  50.884460   4.852883              565   
16669  10823014        2100  51.225700   4.451309       

C:\Users\arian\AppData\Local\Temp\ipykernel_16492\3802197773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=get_column_missing_values(percentage_missing_all), inplace=True)  # Modify DataFrame inplace
C:\Users\arian\AppData\Local\Temp\ipykernel_16492\3802197773.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=get_column_missing_values(percentage_missing_all), inplace=True)  # Modify DataFrame inplace


In [22]:
def encode_data(X_train, X_test):
    # Initialize LabelEncoder
    label_encoder = LabelEncoder()

    # Iterate over each column in df_train
    for col in X_train.columns:
        if X_train[col].dtype == 'object':
            X_train[col] = label_encoder.fit_transform(X_train[col])
            X_test[col] = X_test[col].map(lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1)

    return X_train, X_test

# Encoding categorical features for df_house
X_train_house, X_test_house = encode_data(X_train_house, X_test_house)
print(type(X_train_house))

# Encoding categorical features for df_apartment
X_train_apartment, X_test_apartment = encode_data(X_train_apartment, X_test_apartment)
print(type(X_train_apartment))

print(X_train_house.head())
print(X_train_apartment.head())

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
               id  region  province  district  locality  postalcode  \
9338   11079012.0       2         6        19       488      4032.0   
61289  11041339.0       2         6        42       825      4250.0   
14921  10805399.0       1        10        28       400      8450.0   
13162  11136139.0       1         5        14      1003      3500.0   
29224  11071911.0       1         0        22       350      2880.0   

        latitude  longitude  type  subtype  ...  bedrooms     surface  \
9338   50.614829   5.612938     0        0  ...       4.0  237.000000   
61289  50.668047   5.187740     0        6  ...       3.0  193.240886   
14921  51.252348   2.985051     0       12  ...       2.0  212.000000   
13162  50.967183   5.358411     0        0  ...       2.0  159.000000   
29224  51.094303   4.239369     0        6  ...       3.0  205.000000   

       surfaceGood  hasGasWaterElectricityConnection  cond

In [23]:
def standardize_data(X_train, X_test):
    # Assuming you have a list of column names (feature names) for X_train and X_test
    column_names = X_train.columns.tolist()

    # Initialize StandardScaler
    scaler = StandardScaler()

    # Fit the scaler to the training data and transform both training and test sets
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Convert the scaled arrays back into DataFrames
    X_train = pd.DataFrame(X_train, columns=column_names)
    X_test_ = pd.DataFrame(X_test, columns=column_names)

    return X_train, X_test
print(type(X_train_apartment))
# Standardizing data for df_house
X_train_house, X_test_house = standardize_data(X_train_house, X_test_house )

# Standardizing data for df_apartment
X_train_apartment, X_test_apartment = standardize_data(X_train_apartment, X_test_apartment)
print(X_train_house.head())
print(X_train_apartment.head())

<class 'pandas.core.frame.DataFrame'>
         id    region  province  district  locality  postalcode  latitude  \
0  0.399717  1.359404  0.499096  0.074458 -1.054335   -0.562078 -0.675965   
1  0.250408  1.359404  0.499096  2.006126 -0.634875   -0.485910 -0.513835   
2 -0.684692 -0.592903  1.718037  0.830328 -1.163867    0.981550  1.266254   
3  0.626128 -0.592903  0.194361 -0.345469 -0.413321   -0.747956  0.397491   
4  0.371574 -0.592903 -1.329315  0.326415 -1.226101   -0.964581  0.784766   

   longitude  type   subtype  ...  bedrooms   surface  surfaceGood  \
0   1.517520   0.0 -2.853654  ...  0.508857  0.355902    -0.065198   
1   0.991094   0.0 -0.112513  ... -0.216041  0.000000    -0.024806   
2  -1.735993   0.0  2.628629  ... -0.940939  0.152572    -0.012271   
3   1.202398   0.0 -2.853654  ... -0.940939 -0.278489     0.249860   
4  -0.183056   0.0 -0.112513  ... -0.216041  0.095639    -0.077873   

   hasGasWaterElectricityConnection  condition   facadeCount  hasKitchenSetup 

In [24]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Train a Linear Regression model for df_house
model_house = LinearRegression()
model_house.fit(X_train_house, y_train_house)

# Predict on the training set for df_house
y_train_pred_house = model_house.predict(X_train_house)

# Calculate training R-squared score for df_house
train_r2_house = r2_score(y_train_house, y_train_pred_house)
print("Training R-squared score for df_house:", train_r2_house)

# Predict on the testing set for df_house
y_pred_house = model_house.predict(X_test_house)

# Calculate R-squared score for df_house
r2_house = r2_score(y_test_house, y_pred_house)
print("R-squared score for df_house:", r2_house)

# Train a Linear Regression model for df_apartment
model_apartment = LinearRegression()
model_apartment.fit(X_train_apartment, y_train_apartment)

# Predict on the training set for df_apartment
y_train_pred_apartment = model_apartment.predict(X_train_apartment)

# Calculate training R-squared score for df_apartment
train_r2_apartment = r2_score(y_train_apartment, y_train_pred_apartment)
print("Training R-squared score for df_apartment:", train_r2_apartment)

# Predict on the testing set for df_apartment
y_pred_apartment = model_apartment.predict(X_test_apartment)

# Calculate R-squared score for df_apartment
r2_apartment = r2_score(y_test_apartment, y_pred_apartment)
print("R-squared score for df_apartment:", r2_apartment)

Training R-squared score for df_house: 0.45534173013987544
R-squared score for df_house: 0.453114647876141
Training R-squared score for df_apartment: 0.35933422538736914
R-squared score for df_apartment: 0.09145885066636728


C:\Users\arian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\arian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

# Train a Random Forest Regressor for df_house
model_house = RandomForestRegressor(random_state=42)
model_house.fit(X_train_house, y_train_house)

# Predict on the training set for df_house
y_train_pred_house = model_house.predict(X_train_house)

# Calculate training R-squared score for df_house
train_r2_house = r2_score(y_train_house, y_train_pred_house)
print("Training R-squared score for df_house:", train_r2_house)

# Predict on the testing set for df_house
y_pred_house = model_house.predict(X_test_house)

# Calculate R-squared score for df_house
r2_house = r2_score(y_test_house, y_pred_house)
print("R-squared score for df_house:", r2_house)

# Train a Random Forest Regressor for df_apartment
model_apartment = RandomForestRegressor(random_state=42)
model_apartment.fit(X_train_apartment, y_train_apartment)

# Predict on the training set for df_apartment
y_train_pred_apartment = model_apartment.predict(X_train_apartment)

# Calculate training R-squared score for df_apartment
train_r2_apartment = r2_score(y_train_apartment, y_train_pred_apartment)
print("Training R-squared score for df_apartment:", train_r2_apartment)

# Predict on the testing set for df_apartment
y_pred_apartment = model_apartment.predict(X_test_apartment)

# Calculate R-squared score for df_apartment
r2_apartment = r2_score(y_test_apartment, y_pred_apartment)
print("R-squared score for df_apartment:", r2_apartment)

Training R-squared score for df_house: 0.9625496404193495
R-squared score for df_house: 0.7296044996022564


C:\Users\arian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


Training R-squared score for df_apartment: 0.9598453411964258
R-squared score for df_apartment: 0.7166429526436202


C:\Users\arian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [26]:
# Get the coefficients for X_train_apartment
coefficients_apartment = model_apartment.feature_importances_

# Print the coefficients along with corresponding feature names if available
if hasattr(X_train_apartment, 'columns'):
    print("Feature Importances for Apartment Model:")
    for feature, coef in zip(X_train_apartment.columns, coefficients_apartment):
        print(feature, ':', coef)
else:
    print("Feature Importances for Apartment Model:", coefficients_apartment)

# Get the coefficients for X_train_house
coefficients_house = model_house.feature_importances_

# Print the coefficients along with corresponding feature names if available
if hasattr(X_train_house, 'columns'):
    print("\nFeature Importances for House Model:")
    for feature, coef in zip(X_train_house.columns, coefficients_house):
        print(feature, ':', coef)
else:
    print("\nFeature Importances for House Model:", coefficients_house)

Feature Importances for Apartment Model:
id : 0.042004591621171296
region : 0.0017939880547583485
province : 0.010423813447870047
district : 0.017482713636604946
locality : 0.03037721652883747
postalcode : 0.05929704417543782
latitude : 0.15394749372011995
longitude : 0.07945382163858901
type : 0.0
subtype : 0.012697722449519035
heatingType : 0.004707506805525737
cadastralIncome : 0.042088820937797686
epcScores : 0.01968629654753008
primaryEnergyConsumption : 0.10090890148266199
bedrooms : 0.023206103362314705
surface : 0.3513856153262393
condition : 0.022849459139457233
facadeCount : 0.010100955698452208
hasKitchenSetup : 0.013891646587880644
fireplaceExists : 0.0026289021654047886
floodZone : 0.0010673866738278093
isNewRealEstateProject : 0.0

Feature Importances for House Model:
id : 0.026972233496885916
region : 0.0027312447764497837
province : 0.010053550368570427
district : 0.0176564463344756
locality : 0.022745220782316004
postalcode : 0.0656385016866941
latitude : 0.14425859686

In [27]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import GridSearchCV, cross_val_score

def train_and_evaluate(X_train, y_train, X_test, y_test):
    # Define hyperparameters grid for Random Forest
    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 5],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1]
    }

    # Perform grid search with cross-validation
    rf_regressor = RandomForestRegressor()
    grid_search = GridSearchCV(rf_regressor, param_grid, cv=3, scoring='r2')
    grid_search.fit(X_train, y_train)

    # Print the best parameters for the model
    print("Best parameters:", grid_search.best_params_)

    # Get the best model
    best_model = grid_search.best_estimator_

    # Evaluate the best model on the test data
    test_score = best_model.score(X_test, y_test)
    print("R-squared score on test data:", test_score)

    # Return the best model
    return best_model

# Train and evaluate for df_house
print("House Model Evaluation:")
best_house_model = train_and_evaluate(X_train_house, y_train_house, X_test_house, y_test_house)


House Model Evaluation:
Best parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


C:\Users\arian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


R-squared score on test data: 0.7305301842323355
